In [64]:
#Cameron R. Shabahang
#Risk Management

import pandas as pd
import numpy as np
import scipy.stats as stat
import matplotlib.pyplot as plt
import math
import scipy as sp
from sklearn.decomposition import PCA

#Choose 10 stocks and 1 year historical daily prices
df1 = pd.read_csv('JPM.csv', dtype = object)
df2 = pd.read_csv('GS.csv', dtype = object)
df3 = pd.read_csv('MS.csv', dtype = object)
df4 = pd.read_csv('C.csv', dtype = object)
df5 = pd.read_csv('CS.csv', dtype = object)
df6 = pd.read_csv('UBS.csv', dtype = object)
df7 = pd.read_csv('JEF.csv', dtype = object)
df8 = pd.read_csv('LAZ.csv', dtype = object)
df9 = pd.read_csv('BCS.csv', dtype = object)
df10 = pd.read_csv('EVR.csv', dtype = object)

#Form a portfolio with chosen weights and shares
def filter(df):
    z = df[['Date','Adj Close']]
    close = z['Adj Close'].astype(float)
    dailyreturns = close.pct_change(1)
    z.insert(2,"Daily Return",dailyreturns)
    z['Daily Px Change'] = close - close.shift(1)
    z = z.iloc[1:]
    z['Weighted'] = z['Daily Return'].apply(lambda x: x*0.1)
    return z

df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = [filter(df) for df in (df1, df2, df3, df4, df5, df6, df7, df8, df9, df10)]
portfolio_return = df1['Weighted']+df2['Weighted']+df3['Weighted']+df4['Weighted']+df5['Weighted']+ df6['Weighted']+df7['Weighted']+df8['Weighted']+df9['Weighted']+df10['Weighted']
print(portfolio_return)

#Calculate 99% historical VaR for the portfolio
portfolioVaRobservation = round(0.01 * len(portfolio_return))
portfoliohistoricalVaR = sorted(portfolio_return)[portfolioVaRobservation]*100
print('The 99% historical VaR is:',portfoliohistoricalVaR,'%')



1     -0.008636
2     -0.005588
3     -0.000178
4     -0.003326
5      0.009783
         ...   
248   -0.032210
249   -0.003011
250   -0.047456
251   -0.015656
252    0.045301
Name: Weighted, Length: 252, dtype: float64
The 99% historical VaR is: -9.204044592969312 %


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [200]:
#Calculate 99% parametric VaR for the portfolio
def pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10):
    weights = np.repeat(0.10, 252)
    #df_wMu = 0.1*((np.mean(df1['Daily Return'])+np.mean(df2['Daily Return'])+np.mean(df3['Daily Return'])+np.mean(df4['Daily Return'])+ #Needs to take df1-df10
                  #np.mean(df4['Daily Return'])+np.mean(df5['Daily Return'])+np.mean(df6['Daily Return'])+np.mean(df7['Daily Return'])+
                  #np.mean(df8['Daily Return'])+np.mean(df9['Daily Return'])+np.mean(df10['Daily Return']))
    frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
    df_merge = pd.concat(frames, axis=1)
    z=df_merge['Daily Return']
    print(z)
    covar = np.cov(df_merge['Daily Return'])
    pvar = np.dot(weights.T,np.dot(covar,weights)) #Takes covariance of df1-df10
    portfolioParametricVaR = 2.326*np.sqrt(pvar)
    return portfolioParametricVaR
print('The 99% parametric portfolio VaR is:', pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10))



     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  \
1       -0.007349     -0.009572     -0.014712     -0.009666     -0.011765   
2        0.001234     -0.006362     -0.005468     -0.008325     -0.002976   
3        0.001232     -0.002248      0.001269      0.001592     -0.010448   
4       -0.001670     -0.017635      0.000634     -0.005347     -0.004525   
5        0.000528      0.004288     -0.001900     -0.001453      0.008333   
..            ...           ...           ...           ...           ...   
248     -0.044473     -0.027570     -0.036514     -0.015398     -0.024803   
249     -0.027396     -0.005302      0.007074     -0.026992      0.009249   
250     -0.049319      0.001627     -0.036628     -0.056363     -0.073310   
251     -0.038110     -0.008290     -0.001042     -0.054596     -0.032138   
252      0.061834      0.008953      0.000261      0.083045      0.047893   

     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  

In [212]:
#Portfolio 0.95 decay SD for VaR

#Portfolio 0.95 decay SD for VaR
decay_mean = np.zeros(252)
decay_mean_shift = np.zeros(252)
decay_var = np.zeros(252)
for i in range (len(portfolio_return)):
    if i == 0:
        decay_var[i] = 0
    elif i == 1:
        rollingreturn = portfolio_return[0:i + 1]
        rollingreturn_shift = portfolio_return[0:i]
        decay_mean[i] = np.mean(rollingreturn)
        decay_mean_shift[i] = np.mean(rollingreturn_shift)
        decay_var[i] = 0.95 * decay_var[i - 1]+(1 - 0.95) * (rollingreturn_shift[i] - decay_mean[i]) ** 2
    else:
        rollingreturn = portfolio_return[0:i + 1]
        rollingreturn_shift = portfolio_return[0:i]
        decay_mean[i] = np.mean(rollingreturn)
        decay_mean_shift[i] = np.mean(rollingreturn_shift)
        decay_var[i] = 0.95 * decay_var[i - 1]+(1 - 0.95) * (rollingreturn_shift[i] - decay_mean[i]) ** 2
print('The 0.95 decay VaR is: ', np.sqrt(decay_var[251]))

The 0.95 decay VaR is:  0.055643821950066015


In [181]:
#Calculate 99% historical VaR for the portfolio
portfolioVaRobservation = round(0.01 * len(portfolio_return))
portfoliohistoricalVaR = sorted(portfolio_return)[portfolioVaRobservation]*100
print('The 99% historical VaR is:',portfoliohistoricalVaR,'%')


#Calculate 99% historical PnL VaR for the portfolio
positionsize = 100000
sharesize = positionsize/(df1)
historicalportfolioPnLVaR = sorted(df1['Daily Px Change'])[VaRobservation]*sharesize + sorted(df['Daily Px Change'])[VaRobservation]*sharesize_c
print('The 95% historical portfolio PnL VaR is: $', historicalportfolioPnLVaR)

#Calculate 99% parametric PnL VaR for the portfolio


The 99% historical VaR is: -9.204044592969312 %


TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [187]:
#Implement Factor Model VaR
#Run principle component analysis and retrieve the first two factors
#95% of the eigenvalues
#Locate the 1% position 

import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification

X = portfolio_return
X, y = make_classification(n_samples=1000)
n_samples = X.shape[0]

pca = PCA()
X_transformed = pca.fit_transform(X)

# We center the data and compute the sample covariance matrix.
X_centered = X - np.mean(X, axis=0)
cov_matrix = np.dot(X_centered.T, X_centered) / n_samples
eigenvalues = pca.explained_variance_
count = 0
for eigenvalue, eigenvector in zip(eigenvalues, pca.components_):    
    print('\n', np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector)))
    print(eigenvalue)
    count = count+1
print('\n', count)




 3.267127985292776
3.270398383676455

 2.0818400848748433
2.0839240088837214

 1.1958920862691855
1.197089175444631

 1.1735896755185315
1.1747644399584884

 1.1526517295261203
1.153805535061179

 1.11804877466195
1.1191679426045573

 1.0841135170435923
1.0851987157593486

 1.0770544846358014
1.0781326172530552

 1.0548466276666713
1.0559025301968692

 0.999922391554511
1.0009233148693806

 0.9770229708194749
0.9780009717912657

 0.9598423350263157
0.9608031381644804

 0.9181524733598827
0.9190715449047878

 0.8989326603835832
0.8998324928764582

 0.8916894909010108
0.8925820729739846

 0.8648738758567971
0.8657396154722684

 0.8441278652148076
0.8449728380528605

 0.7676802022124971
0.7684486508633602

 -5.928872630109814e-17
8.56365500341754e-32

 2.6730031397264445e-17
5.927106744100431e-33

 20


In [118]:
#HW2 EVT, EWMA(GARCH), and MC for Parametric Portfolio VaR
#EVT in VaR
#Find values for u and nu
port_SD = pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10)/2.326
u = 1.645*port_SD
order = portfolio_return.sort_values(ascending=False)
order = order.reset_index(drop=True)
print(order)   
i=0
while(order[i]>u):
    i=i+1
evt = order[0:i]
print('EVT list is:\n', evt)
n = len(portfolio_return)
a = 0.05
nu = i
print('The value of u is: ', u)
print('The value of nu is: ', nu)



     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  \
1       -0.007349     -0.009572     -0.014712     -0.009666     -0.011765   
2        0.001234     -0.006362     -0.005468     -0.008325     -0.002976   
3        0.001232     -0.002248      0.001269      0.001592     -0.010448   
4       -0.001670     -0.017635      0.000634     -0.005347     -0.004525   
5        0.000528      0.004288     -0.001900     -0.001453      0.008333   
..            ...           ...           ...           ...           ...   
248     -0.044473     -0.027570     -0.036514     -0.015398     -0.024803   
249     -0.027396     -0.005302      0.007074     -0.026992      0.009249   
250     -0.049319      0.001627     -0.036628     -0.056363     -0.073310   
251     -0.038110     -0.008290     -0.001042     -0.054596     -0.032138   
252      0.061834      0.008953      0.000261      0.083045      0.047893   

     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  

In [178]:
from scipy.optimize import minimize
#Equation 10.9, p. 156
#Estimate xi and beta
xi = 0
beta = 0
def lik(beta, xi):
    y=(n/nu)*(1+(xi*(evt-u)/beta))**(-1/(xi))
    return(-(1/beta)*sum(np.log(y)))
lik_model = minimize(lik,[1.5,1.5], args =(xi),  method='SLSQP')
print(lik_model)



ValueError: operands could not be broadcast together with shapes (18,) (2,) 

In [ ]:
#Solve for u*
ustar = u+(beta/xi)*(((n/nu)*(a))**(-xi)-1)
print(ustar)

#Simulate 1 day distribution
pca=princomp(returns)
eigvals
explained=np.zeros(len(eigvals))
for(i in range 1:len(eigvals))
  explained[i]=eigvals[i]/sum(eigvals)

#95% test
n95=0
e95=0
while(esum95<0.95):
    esum95=esum95+explained[n95+1]
    n95=n95+1

#6 factors required for 95%
varcov<-cov(pca$loadings)
facmean<-mean(pca$loadings)


In [191]:
#2 Days of Prices with EWMA
#window = 60 or 3 months
lam=0.97
n=2
set.seed(-1000)
errors=rnorm(1000, 0, 1)
rets=np.zeros(len(n+1))
prices=np.zeros(len(n+1))
sigsq=np.zeros(len(n+1))
eps=np.zeros(N=1)
rets[1]=mean(portRet) #set to simulated mean
prices[1]=1000000
sigsq[1]=sd(portRet)**2 #set to simulated vol
eps[1]=sqrt(sigsq)*errors[1]
for(i in range 2:(n+1)):
  sigsq[i]=lam*sigsq[i-1]**2+(1-lam)*rets[i-1]**2
  eps[i]=np.sqrt(sigsq[i])*errors[i]
  rets[i]=eps[i]+lam*rets[i-1]+(1-lam)*eps[i-1]
  prices[i]=(1+rets[i])*prices[i-1])

SyntaxError: invalid syntax (<ipython-input-191-89e4a54345f9>, line 15)